## Introduction: Building Molecules

This exercise introduces you to the basic workflow of computational chemistry using the Atomic Simulation Environment (ASE) and NGLView within a Jupyter Notebook. Your goal is to build molecules, manage their data (saving them in the XYZ format), and visualize them.

### Exersices:

1. Builing molecules using ASE
2. Create a scientific plot

### Exercise 1: Building molecules

In the following, import all necessary packages and libraries. If a package is not installed, use 'pip install package' 

ASE is essential for handling atomic structures, and NGLView allows for interactive 3D visualization.

In [1]:
import numpy as np
from ase import Atoms 
from ase.build import molecule
from ase.data.pubchem import pubchem_atoms_search
from ase.visualize import view
from ase.io import write
import nglview as nv

In this exersice, you will learn how to load molecular structures using the ASE package, save them as xyz files, and visualize them interactively using nglview and ase view.

#### Example with water: 
create a water molecule using ASE

In [2]:
# load molecular structure using the molecule function from ASE
water = molecule("H2O")

Print information about the system, such as the chemical symbols, chemical formula, positions of the atoms, atomic numbers, distances between the atoms H and O, and the H-O-H angle. All this is possible using functions implemented in ASE. You can use the documentation to find the functions: https://ase-lib.org/index.html

Hint: all the functions start with get_ ...

In [3]:
# print chemical formula
print(water.get_chemical_formula())

# print chemical symbols
print(water.get_chemical_symbols())

# print positions of the atoms
pos = np.array(water.get_positions())
print(pos)

# print atomic numbers
print(water.get_atomic_numbers())

# print distances between the atoms H and O
print(np.linalg.norm(pos[0] - pos[1]))

# print the H-O-H angle
print(water.get_angle(1, 0, 2))

H2O
['O', 'H', 'H']
[[ 0.        0.        0.119262]
 [ 0.        0.763239 -0.477047]
 [ 0.       -0.763239 -0.477047]]
[8 1 1]
0.9685650182625842
103.99987509868838


Save the molecule as an xyz file by writing it to a file called "water.xyz".

In [4]:
# save the molecular structure as xyz file
write("water.xyz", water)

Visualize the molecule to make sure that it looks correct. You can use the nglview package for visualization. Use the function show_ase() from the package nglview which was imported as nv.

In [5]:
# visualize the molecule
view = nv.show_ase(water)
view

NGLWidget()

You can also save the picture of the molecule by using the function download_image() from the nglview package. Save the image as filename = "water.png".

In [6]:
# save the picture of the molecule as png file
view.download_image(filename="water.png")

Create the objects for the molecules isobutane and acetone and visualize them.

In [7]:
# Isobutane
isobutane = molecule("isobutane")
iso_view = nv.show_ase(isobutane)
iso_view

NGLWidget()

In [8]:
# Acetone
acetone = molecule("CH3COCH3")
ace_view = nv.show_ase(acetone)
ace_view

# 

NGLWidget()

In [32]:
# Calculate structural parameters for acetone
import numpy as np

# Get atomic positions and symbols
positions = acetone.get_positions()
symbols = acetone.get_chemical_symbols()

print("Atom indices and symbols:")
for i, sym in enumerate(symbols):
    print(f"  Atom {i}: {sym} at position ({positions[i][0]:.3f}, {positions[i][1]:.3f}, {positions[i][2]:.3f}) Å")

# Function to calculate bond length
def bond_length(pos1, pos2):
    return np.linalg.norm(pos1 - pos2)

# Function to calculate bond angle (in degrees)
def bond_angle(pos1, pos2, pos3):
    """Calculate angle at pos2 formed by pos1-pos2-pos3"""
    v1 = pos1 - pos2
    v2 = pos3 - pos2
    cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return np.degrees(np.arccos(np.clip(cos_angle, -1.0, 1.0)))

# Function to calculate dihedral angle (in degrees)
def dihedral_angle(pos1, pos2, pos3, pos4):
    """Calculate dihedral angle for four atoms"""
    b1 = pos2 - pos1
    b2 = pos3 - pos2
    b3 = pos4 - pos3
    
    n1 = np.cross(b1, b2)
    n2 = np.cross(b2, b3)
    
    m1 = np.cross(n1, b2 / np.linalg.norm(b2))
    
    x = np.dot(n1, n2)
    y = np.dot(m1, n2)
    
    return np.degrees(np.arctan2(y, x))

Atom indices and symbols:
  Atom 0: O at position (0.000, 0.000, 1.406) Å
  Atom 1: C at position (0.000, 0.000, 0.179) Å
  Atom 2: C at position (0.000, 1.285, -0.616) Å
  Atom 3: C at position (0.000, -1.285, -0.616) Å
  Atom 4: H at position (0.000, 2.135, 0.067) Å
  Atom 5: H at position (0.000, -2.135, 0.067) Å
  Atom 6: H at position (-0.881, 1.332, -1.264) Å
  Atom 7: H at position (0.881, 1.332, -1.264) Å
  Atom 8: H at position (0.881, -1.332, -1.264) Å
  Atom 9: H at position (-0.881, -1.332, -1.264) Å


In [33]:
# Interesting bond lengths (No C-H length)

# Find carbon and oxygen atoms
carbon_indices = [i for i, sym in enumerate(symbols) if sym == 'C']
oxygen_indices = [i for i, sym in enumerate(symbols) if sym == 'O']

# C=O bond (carbonyl)
if len(oxygen_indices) > 0:
    # Find which carbon is closest to oxygen (carbonyl carbon)
    distances_to_O = [(i, bond_length(positions[i], positions[oxygen_indices[0]])) 
                      for i in carbon_indices]
    carbonyl_C = min(distances_to_O, key=lambda x: x[1])[0]
    co_bond = bond_length(positions[carbonyl_C], positions[oxygen_indices[0]])
    print(f"C=O bond length (carbonyl): {co_bond:.3f} Å")
    
    # C-C bonds
    other_carbons = [c for c in carbon_indices if c != carbonyl_C]
    for c in other_carbons:
        cc_bond = bond_length(positions[carbonyl_C], positions[c])
        print(f"C-C bond length (C{carbonyl_C}-C{c}): {cc_bond:.3f} Å")

C=O bond length (carbonyl): 1.227 Å
C-C bond length (C1-C2): 1.512 Å
C-C bond length (C1-C3): 1.512 Å


In [34]:
# Calculate bond angles in acetone

# C-C=O angle (at carbonyl carbon)
if len(oxygen_indices) > 0 and len(other_carbons) >= 1:
    angle_CCO = bond_angle(positions[other_carbons[0]], 
                           positions[carbonyl_C], 
                           positions[oxygen_indices[0]])
    print(f"C-C=O angle: {angle_CCO:.2f}°")
    
    # C-C-C angle (at carbonyl carbon) - between the two methyl groups
    if len(other_carbons) >= 2:
        angle_CCC = bond_angle(positions[other_carbons[0]], 
                               positions[carbonyl_C], 
                               positions[other_carbons[1]])
        print(f"C-C-C angle (between methyl groups): {angle_CCC:.2f}°")
    
    # H-C-H angles in methyl groups
    for c in other_carbons[:1]:  # Just show one methyl group as example
        h_near_c = [h for h in h_indices if bond_length(positions[c], positions[h]) < 1.2]
        if len(h_near_c) >= 2:
            angle_HCH = bond_angle(positions[h_near_c[0]], 
                                   positions[c], 
                                   positions[h_near_c[1]])
            print(f"H-C-H angle in methyl group: {angle_HCH:.2f}°")
    
    # C-C-H angle (methyl group)
    for c in other_carbons[:1]:
        h_near_c = [h for h in h_indices if bond_length(positions[c], positions[h]) < 1.2]
        if h_near_c:
            angle_CCH = bond_angle(positions[carbonyl_C], 
                                   positions[c], 
                                   positions[h_near_c[0]])
            print(f"C-C-H angle in methyl group: {angle_CCH:.2f}°")

C-C=O angle: 121.75°
C-C-C angle (between methyl groups): 116.51°
H-C-H angle in methyl group: 109.75°
C-C-H angle in methyl group: 109.46°


In [35]:
# Calculate dihedral angles in acetone
    
# C-C-C=O dihedral
dihedral_CCCO = dihedral_angle(positions[other_carbons[0]], positions[other_carbons[1]], positions[carbonyl_C], positions[oxygen_indices[0]])
print(f"C-C-C=O dihedral angle: {dihedral_CCCO:.2f}°")

C-C-C=O dihedral angle: 180.00°


Some molecules cannot be created using the molecule() function from ase for example for the molecules benzophenone, maleicacid and 2,4,6-trinitrophenol. Instead, there is a function from ase.data.pubchem which is called pubchem_atoms_search(). Using this, you can generate the structures and visualize them.

In [9]:
# Benzophenone
benzo = pubchem_atoms_search("Benzophenone")
benzo_view = nv.show_ase(benzo)
benzo_view

NGLWidget()

In [10]:
# 2,4,6-trinitrophenol
trini = pubchem_atoms_search("2,4,6-trinitrophenol")
trini_view = nv.show_ase(trini)
trini_view

NGLWidget()

In [11]:
# Maleicacid
male = pubchem_atoms_search("Maleicacid")
male_view = nv.show_ase(male)
male_view

/home/tolletafel/Documents/CompChem/venv/lib/python3.12/site-packages/ase/data/pubchem.py:74: UserWarning: The structure "Maleicacid" has more than one conformer in PubChem. By default, the first conformer is returned, please ensure you are using the structure you intend to or use the `ase.data.pubchem.pubchem_conformer_search` function
  warnings.warn(


NGLWidget()

Some molecules are not available in the data base. Instead, you can enter the positions of the atoms manually. For example, for tetrachloroplatinate, you can use the following coordinates:

x, y and z coordinates in Angstroms:

Pt 0.0 0.0 0.0

Cl 2.3 0.0 0.0

Cl -2.3 0.0 0.0

Cl 0.0 2.3 0.0

Cl 0.0 -2.3 0.0

In [12]:
tetra = Atoms(["Pt", "Cl", "Cl", "Cl", "Cl"], positions=[(0, 0, 0), (2.3, 0, 0), (-2.3, 0, 0), (0, 2.3, 0), (0, -2.3, 0)])
tetra_view = nv.show_ase(tetra)
tetra_view

NGLWidget()

### Exercise 2: Creating a scientific plot

Load the provided datafile /software/uebungen/templates/Aufgabe1/morse-potential.dat and create a plot the corresponding data. 

First, import the packages numpy and matplotlib.pyplot.

In [13]:
import numpy as np
import matplotlib.pyplot as plt

Then, load the data using the loadtxt() function from numpy. The data file contains two columns: the first column is the distance between two atoms and the second column is the potential energy.

In [14]:
# load data
data =  np.loadtxt("morse-potential.dat")

FileNotFoundError: morse-potential.dat not found.

Finally, create a plot of the potential energy as a function of atom distance using matplotlib. Make sure to label your axes properly. The x-axis should be labeled "Distance (Å)" and the y-axis should be labeled "Potential Energy (eV)". You can also add a title to your plot, such as "Morse Potential".

In [ ]:
# create plot
plt.plot(data.T[0], data.T[1])
plt.xlabel(r"Distance ($\AA$)")
plt.ylabel("Potential Energy (eV)")
plt.title("Morse Potential")
plt.show()